In [ ]:
#!g1.1
import torch
from torch import nn
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import clear_output
from tqdm.notebook import tqdm
import os
import pandas as pd
import numpy as np
from torchvision.io import read_image
from torch.utils.data import Dataset
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, indices=None):
        self.img_labels = pd.read_csv(annotations_file)
        if indices is not None:
            self.img_labels = self.img_labels.loc[indices,:]
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path).type(torch.FloatTensor).to(device)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        return image, label

In [ ]:
#!g1.1
split_df = pd.read_csv("/kaggle/input/bhw-1-deep-learning/bhw1-dataset/labels.csv")
train_idx = np.array([])
test_idx = np.array([])
for i in range(200):
    tmp = np.random.permutation(np.array(split_df[split_df["Label"] == i].index.tolist()))
    train_idx = np.append(train_idx, tmp[:490])
    test_idx = np.append(test_idx, tmp[490:500])

In [ ]:
#!g1.1
from torch.utils.data import DataLoader, random_split
import torchvision.transforms as T
normalize = T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
train_transform = T.Compose([
    T.RandomResizedCrop(224, scale=(0.5, 1.0)),
    T.RandomHorizontalFlip(p=0.5),
    T.GaussianBlur(3, (0.1, 4)),
    T.RandomInvert(p=0.05),
    T.RandomGrayscale(p=0.1),
    normalize,
])
test_transform = T.Compose([
    T.Resize(256),
    T.CenterCrop(224),
    normalize,
])
train_dataset = CustomImageDataset("/kaggle/input/bhw-1-deep-learning/bhw1-dataset/labels.csv", 
                                   "/kaggle/input/bhw-1-deep-learning/bhw1-dataset/trainval", 
                                   transform=train_transform, 
                                   indices=train_idx)

val_dataset = CustomImageDataset("/kaggle/input/bhw-1-deep-learning/bhw1-dataset/labels.csv", 
                                   "/kaggle/input/bhw-1-deep-learning/bhw1-dataset/trainval", 
                                   transform=test_transform, 
                                   indices=test_idx)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=False)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, pin_memory=False)

In [ ]:
#!g1.1
sns.set_style('whitegrid')
plt.rcParams.update({'font.size': 15})


def plot_losses(train_losses, test_losses, train_accuracies, test_accuracies):
    clear_output()
    fig, axs = plt.subplots(1, 2, figsize=(13, 4))
    axs[0].plot(range(1, len(train_losses) + 1), train_losses, label='train')
    axs[0].plot(range(1, len(test_losses) + 1), test_losses, label='test')
    axs[0].set_ylabel('loss')

    axs[1].plot(range(1, len(train_accuracies) + 1), train_accuracies, label='train')
    axs[1].plot(range(1, len(test_accuracies) + 1), test_accuracies, label='test')
    axs[1].set_ylabel('accuracy')

    for ax in axs:
        ax.set_xlabel('epoch')
        ax.legend()

    plt.show()
    
def training_epoch(model, optimizer, criterion, train_loader, tqdm_desc):
    train_loss, train_accuracy = 0.0, 0.0
    model.train()
    for images, labels in tqdm(train_loader, desc=tqdm_desc):
        images = images.to(device)  # images: batch_size x num_channels x height x width
        labels = labels.to(device)  # labels: batch_size

        optimizer.zero_grad()
        logits = model(images)  # logits: batch_size x num_classes
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.shape[0]
        train_accuracy += (logits.argmax(dim=1) == labels).sum().item()
    
    train_loss /= len(train_loader.dataset)
    train_accuracy /= len(train_loader.dataset)
    return train_loss, train_accuracy


@torch.no_grad()
def validation_epoch(model, criterion, test_loader, tqdm_desc):
    test_loss, test_accuracy = 0.0, 0.0
    model.eval()
    for images, labels in tqdm(test_loader, desc=tqdm_desc):
        images = images.to(device)  # images: batch_size x num_channels x height x width
        labels = labels.to(device)  # labels: batch_size
        logits = model(images)  # logits: batch_size x num_classes
        loss = criterion(logits, labels)

        test_loss += loss.item() * images.shape[0]
        test_accuracy += (logits.argmax(dim=1) == labels).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy /= len(test_loader.dataset)
    return test_loss, test_accuracy

    
def train(model, optimizer, scheduler, criterion, train_loader, test_loader, num_epochs):
    train_losses, train_accuracies = [], []
    test_losses, test_accuracies = [], []

    for epoch in range(1, num_epochs + 1):
        train_loss, train_accuracy = training_epoch(
            model, optimizer, criterion, train_loader,
            tqdm_desc=f'Training {epoch}/{num_epochs}'
        )
        test_loss, test_accuracy = validation_epoch(
            model, criterion, test_loader,
            tqdm_desc=f'Validating {epoch}/{num_epochs}'
        )

        if scheduler is not None:
            scheduler.step(test_loss)

        train_losses += [train_loss]
        train_accuracies += [train_accuracy]
        test_losses += [test_loss]
        test_accuracies += [test_accuracy]
        plot_losses(train_losses, test_losses, train_accuracies, test_accuracies)

    return train_losses, test_losses, train_accuracies, test_accuracies

In [ ]:
#!g1.1
class TestDataset(Dataset):
    def __init__(self, transform=None):
        self.img_dir = "/kaggle/input/bhw-1-deep-learning/bhw1-dataset/test"
        self.transform = transform
        self.img_labels = sorted(os.listdir("/kaggle/input/bhw-1-deep-learning/bhw1-dataset/test"))

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        
        img_path = os.path.join(self.img_dir, self.img_labels[idx])
        image = read_image(img_path).type(torch.FloatTensor)
        label = self.img_labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

In [ ]:
#!g1.1
from torchvision.models import mobilenet_v2

model = mobilenet_v2(num_classes=200).to(device)

In [ ]:
#!g1.1
num_epochs = 50
optimizer = torch.optim.SGD(model.parameters(), lr=0.045, momentum=0.9, weight_decay=0.00004, nesterov=True)
criterion = torch.nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50)
train_losses, test_losses, train_accuracies, test_accuracies = train(
    model, optimizer, scheduler, criterion, train_loader, val_loader, num_epochs
)

In [ ]:
num_epoch = 5
optimizer = torch.optim.SGD(model.parameters(), lr=0.004, momentum=0.9, weight_decay=0.00004, nesterov=True)
criterion = torch.nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)
train_losses, test_losses, train_accuracies, test_accuracies = train(
    model, optimizer, scheduler, criterion, train_loader, val_loader, num_epoch
)

In [ ]:
num_epoch = 5
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.00004, nesterov=True)
train_losses, test_losses, train_accuracies, test_accuracies = train(
    model, optimizer, scheduler, criterion, train_loader, val_loader, num_epoch
)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False, pin_memory=True)
num_epoch = 12
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.00004, nesterov=True)
train_losses, test_losses, train_accuracies, test_accuracies = train(
    model, optimizer, scheduler, criterion, train_loader, val_loader, num_epoch
)

In [ ]:
num_epoch = 16
optimizer = torch.optim.SGD(model.parameters(), lr=0.0005, momentum=0.9, weight_decay=0, nesterov=True)
train_losses, test_losses, train_accuracies, test_accuracies = train(
    model, optimizer, scheduler, criterion, train_loader, val_loader, num_epoch
)

In [ ]:
#!g1.1
test_dataset = TestDataset(transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, pin_memory=False)
df = pd.DataFrame(columns=["Id", "Label"])
# model.to("cpu")
model.eval()
for images, labels in tqdm(test_loader):
    images = images.to(device)  # images: batch_size x num_channels x height x width
    labels = labels  # labels: batch_size
    logits = model(images)
    preds = logits.argmax(dim=1)
    # print(labels, preds)
    tmp = pd.DataFrame(data={"Id": labels, "Label": preds.detach().cpu().numpy()})
    df = pd.concat([df, tmp], ignore_index=True)

df.to_csv("MobileNet_77epoch.csv", index=False)